In [2]:
import matplotlib.pyplot as plt
from pennylane import numpy as np
import pennylane as qml

In [10]:
geo_file = "h2.xyz"
symbols, coordinates = qml.qchem.read_structure(geo_file)
hamiltonian, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates)

4

In [4]:
exact_value = -1.136189454088

In [5]:
np.random.seed(0)
init_params = np.random.uniform(low=0, high=2 * np.pi, size=12, requires_grad=True)
max_iterations = 500
step_size = 0.5
conv_tol = 1e-06
epsilon = .01

In [12]:
dev = qml.device("default.qubit", wires=qubits)
hf_state = np.array([1, 1, 0, 0], requires_grad=False)

def ansatz(params, wires=[0, 1, 2, 3]):
    qml.BasisState(hf_state, wires=wires)
    for i in wires:
        qml.RZ(params[3 * i], wires=i)
        qml.RY(params[3 * i + 1], wires=i)
        qml.RZ(params[3 * i + 2], wires=i)
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[2, 0])
    qml.CNOT(wires=[3, 1])

In [8]:
@qml.qnode(dev)
def cost(params):
    ansatz(params)
    return qml.expval(hamiltonian)
print(qml.draw(cost)(init_params))
# print(init_params)
# print(cost(init_params))

0: ─╭BasisState(M0)──RZ(3.45)──RY(4.49)──RZ(3.79)────╭X────┤ ╭<𝓗>
1: ─├BasisState(M0)──RZ(3.42)──RY(2.66)──RZ(4.06)────│──╭X─┤ ├<𝓗>
2: ─├BasisState(M0)──RZ(2.75)──RY(5.60)──RZ(6.05)─╭●─╰●─│──┤ ├<𝓗>
3: ─╰BasisState(M0)──RZ(2.41)──RY(4.97)──RZ(3.32)─╰X────╰●─┤ ╰<𝓗>


In [23]:
opt = qml.GradientDescentOptimizer(step_size)

params = init_params

gd_cost = []

for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(cost, params)
    params = params + np.random.normal(loc=0, scale=epsilon, size=len(params))
    gd_cost.append(prev_energy)
    energy = cost(params)
    conv = np.abs(energy - prev_energy)
    if n % 20 == 0: print("Iteration = {:},  Energy = {:.8f} Ha".format(n, energy))
    if conv <= conv_tol: break

Iteration = 0,  Energy = -0.08997094 Ha
Iteration = 20,  Energy = -0.53417482 Ha
Iteration = 40,  Energy = -1.12168512 Ha
Iteration = 60,  Energy = -1.13440918 Ha
Iteration = 80,  Energy = -1.13542509 Ha
Iteration = 100,  Energy = -1.13575449 Ha
Iteration = 120,  Energy = -1.13600557 Ha


In [10]:
hamiltonian = qml.Hamiltonian(np.array(hamiltonian.coeffs, requires_grad=False), hamiltonian.ops)

opt = qml.QNGOptimizer(step_size, lam=0.001, approx="block-diag")

params = init_params
prev_energy = cost(params)
qngd_cost = []

for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(cost, params)
    params = params + np.random.normal(loc=0, scale=epsilon, size=len(params))
    qngd_cost.append(prev_energy)
    energy = cost(params)
    conv = np.abs(energy - prev_energy)
    if n % 4 == 0: print("Iteration = {:},  Energy = {:.8f} Ha".format(n, energy))
    if conv <= conv_tol: break

Iteration = 0,  Energy = -0.32164519 Ha
Iteration = 4,  Energy = -0.46875033 Ha
Iteration = 8,  Energy = -0.85091050 Ha
Iteration = 12,  Energy = -1.13575339 Ha
Iteration = 16,  Energy = -1.13618916 Ha
